In [ ]:
import argparse
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets
from torch.autograd import Variable
from tqdm import tqdm
import matplotlib.pyplot as plt
import time

from train import train_model
from inception import *
from data import *
import numpy as np
from tqdm import tqdm
import PIL.Image as Image

## Parameters

In [ ]:
data_dir = 'bird_dataset'
batch_size = 32
epochs = 25
lr = 0.045
momentum = 0.9
seed = 44876
save_dir = 'experiment'

use_cuda = torch.cuda.is_available()
torch.manual_seed(seed)

## Data

In [ ]:
dataloaders = {
    'train': torch.utils.data.DataLoader(
    datasets.ImageFolder(data_dir + '/train_images/',
                         transform=inceptionv3_train_transforms),
    batch_size=batch_size, shuffle=True, num_workers=4),

    'val': torch.utils.data.DataLoader(
    datasets.ImageFolder(data_dir + '/val_images/',
                         transform=inceptionv3_test_transforms),
    batch_size=batch_size, shuffle=False, num_workers=4)
}

## Model

In [ ]:
model = Inception3(num_classes=1000, aux_logits=True)
state_dict = torch.load('experiment/iNat_2018_InceptionV3.pth.tar')['state_dict']
model.load_state_dict(state_dict)    

in_features = model.fc.in_features
model.fc = nn.Linear(in_features, 20)
model.aux_logits = False

for child, layer in model.named_children():
    if child != 'fc':
        for param in layer.parameters():
            param.requires_grad = False

if use_cuda:
    print('Using GPU')
    model.cuda()
else:
    print('Using CPU')
    

optimizer = optim.RMSprop(
    model.parameters(),
    lr=lr, 
    alpha=0.9, 
    eps=1.0, 
    weight_decay=0.00005
)

criterion = torch.nn.CrossEntropyLoss()
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, 2, 0.94)

## Training

In [ ]:
# Train for 5 epoch the last layer
model, _ = train_model(model, dataloaders, criterion, optimizer, exp_lr_scheduler, 5, False)

# Unfreeze the whole network
for child, layer in model.named_children():
    for param in layer.parameters():
        param.requires_grad = True
        
optimizer = optim.RMSprop(
    model.parameters(),
    lr=0.045, 
    alpha=0.9, 
    eps=1.0, 
    weight_decay=0.00005
)

criterion = torch.nn.CrossEntropyLoss()
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, 2, 0.94)

# Train for 10 epochs the whole network
model, _ = train_model(model, dataloaders, criterion, optimizer, exp_lr_scheduler, 10, False)

In [ ]:
torch.save(model.state_dict(), save_dir + '/inceptionv3_1_09417.pth')

In [ ]:
model = Inception3(num_classes=1000, aux_logits=True)
state_dict = torch.load('experiment/iNat_2018_InceptionV3.pth.tar')['state_dict']
model.load_state_dict(state_dict)    

in_features = model.fc.in_features
model.fc = nn.Linear(in_features, 20)
model.aux_logits = False

model.load_state_dict(torch.load('experiment/inceptionv3_0.9417toretrain.pth'))

model.cuda()
None

In [ ]:
# Unfreeze the whole network
for child, layer in model.named_children():
    for param in layer.parameters():
        param.requires_grad = True

optimizer = optim.RMSprop(
    model.parameters(),
    lr=0.001, 
    alpha=0.9, 
    eps=1.0, 
    weight_decay=0.0001
)

criterion = torch.nn.CrossEntropyLoss()
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, 2, 0.7)

# Train for 10 epochs the whole network
model, val = train_model(model, dataloaders, criterion, optimizer, exp_lr_scheduler, 1, False)